In [4]:
import os
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('..')

import numpy as np
import random
import torch
# %env CUDA_VISIBLE_DEVICES=1

from lib import data_processors, utils

from pytorch_pretrained_bert.modeling import (BertConfig, WEIGHTS_NAME, CONFIG_NAME)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [6]:
from lib.bert import BertForSequenceClassification
from pytorch_pretrained_bert.tokenization import BertTokenizer

from lib.train_eval import train, evaluate

from lib.train_student import train_student  # function for training 

Скачиваем модель-учителя - Bert.

In [7]:
params = {
    'data_dir': '../../data/SST-2',
    'output_dir': '../output',
    'cache_dir': '../model_cache',
    'task_name': 'sst2',
    'bert_model': 'bert-base-uncased',
    'max_seq_length': 128,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'learning_rate': 2e-5,
    'warmup_proportion': 0.1,
    'num_train_epochs': 1,
    'seed': 1331
}

processors = {
    'cola': data_processors.ColaProcessor,
    'mnli': data_processors.MnliProcessor,
    'mrpc': data_processors.MrpcProcessor,
    'sst2': data_processors.SST2Processor
}

num_labels_task = {
    'cola': 2,
    'mnli': 3,
    'mrpc': 2,
    'sst2': 2
}

random.seed(params['seed'])
np.random.seed(params['seed'])
torch.manual_seed(params['seed'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

processor = processors[params['task_name']]()
num_labels = num_labels_task[params['task_name']]
label_list = processor.get_labels()

tokenizer = BertTokenizer.from_pretrained(
    params['bert_model'], do_lower_case=True)

train_examples = processor.get_train_examples(params['data_dir'])
eval_examples = processor.get_dev_examples(params['data_dir'])


# teacher_model = BertForSequenceClassification.from_pretrained(
#     params['bert_model'], cache_dir=params['cache_dir'], num_labels=num_labels).to(device)


# Load a trained model and config that you have fine-tuned
config = BertConfig(os.path.join(params['output_dir'], CONFIG_NAME))
teacher_model = BertForSequenceClassification(config, num_labels=num_labels)
teacher_model.load_state_dict(torch.load(os.path.join(params['output_dir'], WEIGHTS_NAME)))
teacher_model.to(device)

04/06/2019 16:22:17 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/shakhrayv/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInterme

В качестве студента возьмем, например, берта без первых 3 блоков и последних 3ех блоков. Скачиваем берт и обрезаем.

In [8]:
model = BertForSequenceClassification.from_pretrained(
    params['bert_model'], cache_dir=params['cache_dir'], num_labels=num_labels).to(device)

all_layers = model.bert.encoder.layer
model.bert.encoder.layer = all_layers[3:9]

04/06/2019 16:22:22 - INFO - lib.bert -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../model_cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/06/2019 16:22:22 - INFO - lib.bert -   extracting archive file ../model_cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpzlsdi217
04/06/2019 16:22:24 - INFO - lib.bert -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

04/06/2019 16:22:26 - INFO - lib.bert -   Weights of BertForSequenceClassification not initi

Обучаем студента при помощи дистилляции. 

Функция train_student принимает те же параметры, что и обычный train и еще модель-учителя teacher_model.

In [9]:
model, result = train_student(model, teacher_model, train_examples, label_list, params, tokenizer)

Evaluating:   0%|          | 0/2105 [00:00<?, ?it/s]

***** Running training *****
Num examples: 67349
Batch size:   32
Num steps:    2104


Iteration:   0%|          | 0/2105 [00:00<?, ?it/s]../lib/train_student.py:113: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  loss_first = KLDivLoss()(F.log_softmax(logits_model / temperature), F.softmax(teacher_logits / temperature))
../lib/train_student.py:113: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss_first = KLDivLoss()(F.log_softmax(logits_model / temperature), F.softmax(teacher_logits / temperature))
/home/shakhrayv/.virtualenvs/lenas_py3.7/lib/python3.7/site-packages/torch/nn/functional.py:1906: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total los

Epoch: 1


Iteration: 100%|██████████| 2105/2105 [06:10<00:00,  6.13it/s]
